# Iceberg Migrator Installer

This notebook installs the Iceberg Migrator.


## PREREQUISITE: Enable GitHub External Access Integration

- If your account ***does not*** have an External Access Integration for the GitHub API, execute the commands below:
```
CREATE OR REPLACE NETWORK RULE gh_network_rule
MODE = EGRESS
TYPE = HOST_PORT
VALUE_LIST = ('github.com', 'api.github.com');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION gh_access_integration
ALLOWED_NETWORK_RULES = (gh_network_rule)
ENABLED = true;
```
- Once the External Access Integration for the GitHub API as been created (or if one already exists), enable this notebook to use it, by following the steps here: https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-external-access#enable-external-access-integrations-eai.
    - **NOTE:** this step restarts the notebook.



## STEP 1: Initialization

In [ ]:
import base64
import codecs
import os
import pandas as pd
import pickle
import re
import requests
import sqlparse
import streamlit as st

session = get_active_session()

#tag session
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"iceberg-migrator","version":{{"major":1, "minor":2}},"attributes":{{"env":"im_acct","component":"installer","type":"notebook"}}}}'""").collect()

#get current_role
current_role = session.get_current_role().replace('"','')

st.success(f"Session initialized for role: {current_role} 🎉")

## STEP 2: Function definition

In [ ]:
def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)

def get_pickle(path):
    req = requests.get(path)
    if req.status_code == requests.codes.ok:
        req = req.json()
        content = base64.b64decode(req['content']).decode("utf-8")
        return content
    else:
        return 'Content was not found.'
    
def put_to_stage(session, stage, filename, type, content):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)

    if type.lower() == "file":
        f = open(local_file, "w", encoding='utf-8')
        f.write(content)
        f.close()

    if type.lower() == "image":
        with open(local_file, "wb") as f:
            img_data = content.encode()
            f.write(base64.b64decode(img_data))
            f.close()
        
    session.file.put(local_file, f"@{stage}", auto_compress=False, overwrite=True)
    return f"saved {type} {filename} in stage {stage}"

## STEP 3: Get pickle file and decode

In [ ]:
decoded_event_acct_obj = None

install_pickle = get_pickle('https://api.github.com/repos/Snowflake-Labs/emerging-solutions-toolbox/contents/helper-iceberg-migrator/pickles/setup.pickle')

if install_pickle != 'Content was not found.':
    decoded_obj = decode(install_pickle)
    st.success(f"Install Pickle Decoded 🎉")    
else:
    print ('Content was not found.')

## STEP 4: Execute commands from decoded pickle file

In [ ]:
install_flag = False

verbose = st.selectbox("Verbose?", ("Select...", "Y", "N"), index=0,)

setup_list = decoded_obj['setup'].items()
for file_name, file_content in setup_list:
    file_content = str(file_content.decode("utf-8"))
    
    #replace variables
    #repl = {"<MY_ROLE>": f"{current_role}"
    #      }

    #repl = dict((re.escape(k), v) for k, v in repl.items()) 
    #pattern = re.compile("|".join(repl.keys()))
    #file_content = pattern.sub(lambda m: repl[re.escape(m.group(0))], file_content)

    #format file_content
    file_content = sqlparse.format(file_content, strip_comments=True).strip()

    #execute each sql statement
    statements = sqlparse.split(file_content)
    for stmt in statements:
        if verbose != "Select...":
            if verbose == "Y":
                st.code(f"""Statement executed: {stmt}
                """)
            session.sql(stmt).collect()

#Streamlit Main
streamlit_main_list = decoded_obj['streamlit_main'].items()
for file_name, file_content in streamlit_main_list:
    file_content = str(file_content.decode("utf-8"))
    
    #replace variables
    #repl = {"<MY_ROLE>": f"{current_role}"
    #      }
    
    #repl = dict((re.escape(k), v) for k, v in repl.items()) 
    #pattern = re.compile("|".join(repl.keys()))
    #file_content = pattern.sub(lambda m: repl[re.escape(m.group(0))], file_content)

    if verbose != "Select...":
        #put file on stage
        stage = f"ICEBERG_MIGRATOR_DB.STREAMLIT.STREAMLIT"
        put_to_stage(session, stage, file_name, "file", file_content)
        st.code(f"""File: {file_name} placed on stage: {stage}
                """)
        
#Streamlit Env
streamlit_env_list = decoded_obj['streamlit_env'].items()
for file_name, file_content in streamlit_env_list:
    file_content = str(file_content.decode("utf-8"))

    if verbose != "Select...":
        #put file on stage
        stage = f"ICEBERG_MIGRATOR_DB.STREAMLIT.STREAMLIT"
        put_to_stage(session, stage, file_name, "file", file_content)
        st.code(f"""File: {file_name} placed on stage: {stage}
                """)

#Streamlit Utils
streamlit_utils_list = decoded_obj['streamlit_utils'].items()
for file_name, file_content in streamlit_utils_list:
    file_content = str(file_content.decode("utf-8"))

    if verbose != "Select...":
        #put file on stage
        stage = f"ICEBERG_MIGRATOR_DB.STREAMLIT.STREAMLIT/utils"
        put_to_stage(session, stage, file_name, "file", file_content)
        st.code(f"""File: {file_name} placed on stage: {stage}
                """)

#Streamlit Imgs
streamlit_img_list = decoded_obj['streamlit_imgs'].items()
for file_name, file_content in streamlit_img_list:
    file_content = str(file_content.decode("utf-8"))

    if verbose != "Select...":
        #put file on stage
        stage = f"ICEBERG_MIGRATOR_DB.STREAMLIT.STREAMLIT/img"
        put_to_stage(session, stage, file_name, "image", file_content)
        st.code(f"""File: {file_name} placed on stage: {stage}
                """)

    if file_name == list(streamlit_img_list)[-1][0]:
        install_flag = True

if install_flag:
    st.success(f"Iceberg Migrator Installed 🎉")